In [2]:
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline
import joblib
from gensim.models.fasttext import FastText
import pandas as pd
import numpy as np
import re
import difflib
import xlsxwriter
from tqdm.notebook import tqdm
from termcolor import colored, cprint
tqdm.pandas()

In [3]:
joblib.__version__

'1.1.1'

In [4]:
def tfidf_featuring(tfidf, df):   
    '''Преобразование текста в мешок слов'''
    X_tfidf = tfidf.transform(df['text'])
    feature_names = tfidf.get_feature_names_out()
    X_tfidf = pd.DataFrame(X_tfidf.toarray(), columns = feature_names, index = df.index)
    
    return X_tfidf

def sentences_split(text):
    # разбивает текст на предложения
    try:
        # TODO оставить точки после числа (?<![0-9])\.  (?<![0-9])\.(?![0-9])
        # TODO удалить текст до требуюутся, трубуется r'требуются:([^<>]+)'
        #разделение на фразы. игнорировать точку после числа. смайлики - разделители |[^\x00-\x7Fа-яА-Я]
        pattern = r'\;| ,|(?<![0-9])\.|\n|\•|—|-|!|обязанности|требования|условия|[^\x00-\x7Fа-яА-Я]'
        sentences = re.compile(pattern).split(text.lower()) 
        #удаляем не буквы в начале и конце фразы
        pattern = r'^[^а-яА-ЯёЁ]+|[^а-яА-ЯёЁ]+$'
        sentences = [re.sub(pattern, '', sen) for sen in sentences]
        return[sen for sen in sentences if len(sen)>0]
    except:
        return []

In [5]:
def sentences_df(df, part=None):
    # датафрейм для извлечения, part сколько строк извдлекаем
    test_=[]
    idxs = df.index.tolist()
    for idx in idxs[0:part]:
        text = df['responsibilities(Должностные обязанности)'][idx]
        #print(sentences_split(text))
        test_.append(sentences_split(text))
    return test_

In [6]:
def sort_respons(sentences):
    # сортирует колонку обязанности на три разных
    responsibilities = []
    requirements =[]
    terms=[]
    for idx in tqdm(range(0, len(sentences))):
        test_tfidf = tfidf_featuring(tfidf, pd.DataFrame({"text": sentences[idx]}))
        catc_proba = model.predict_proba(test_tfidf)

        temp_ = pd.DataFrame({"text": sentences[idx]})
        temp_['target']= np.argmax(catc_proba, axis=1)
        temp_['proba']= np.amax(catc_proba, axis=1)
        temp_['target']= temp_['target'].replace({0: "Обязанности", 1: 'Требования', 2: 'Условия работы'}, regex=True)
        #пока без обязанностей
        resp= temp_[(temp_['target']=='Обязанности')&(temp_['proba']>=0.75)]['text'].tolist()
        responsibilities.append(". ".join([i.capitalize() for i in resp]))
        req = temp_[(temp_['target']=='Требования')&(temp_['proba']>=0.75)]['text'].tolist()
        requirements.append(". ".join([i.capitalize() for i in req]))
        ter = temp_[(temp_['target']=='Условия работы')&(temp_['proba']>=0.75)]['text'].tolist()
        terms.append(". ".join([i.capitalize() for i in ter]))
        
    return responsibilities, requirements, terms

In [7]:
model = CatBoostClassifier(loss_function='MultiClass', random_state=42)
tfidf = TfidfVectorizer()

In [8]:
# загрузка классификатора
model.load_model('model/model.cbm')
# загрузка векторизатора
tfidf = joblib.load('model/tfidf.pkl')
#model = FastText.load("model/fasttext.model")

In [9]:
data = pd.read_excel('data/Датасет.xlsx', index_col=0)

In [10]:
data

,name(название),specialization(специализация),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия),skills(Ключевые навыки),salary_from,salary_to,object,city,...,updated_by,position,phone,website,email,image,unique_code,city_code,source_id,link_resource
id,,,,,,,,,,,,,,,,,,,,,
9495846,Арматурщик,Арматурщик,Работа на строительных площадках. Очистных соо...,NaN,NaN,NaN,150000,170000,NaN,Димитровград,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-81058502,\N,1,https://hh.ru/vacancy/81058502
9495850,Арматурщик,Арматурщик,Гнутье арматурной стали на механических станка...,NaN,NaN,NaN,130000,150000,NaN,Копейск,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-80163818,\N,1,https://hh.ru/vacancy/80163818
9495851,Арматурщик,Арматурщик,Вязка арматуры.,NaN,NaN,NaN,85000,90000,NaN,Ярославль,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-77729491,\N,1,https://hh.ru/vacancy/77729491
2,Арматурщик,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,NaN,\N,50000,100000,"ООО ""АЛЬМИС_ИНТЕГРАЛ""",\N,...,1,Арматурщик,74959214241,https://almisintegral.ru/,info@almisintegral.ru,\N,\N,\N,1,\N
9388857,Арматурщик,Арматурщик Вахта в мcк (питание+проживание) 60/30,Вахта в город Москва. Обязанности: - армирова...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,NaN,67000,134000,NaN,Кировск,...,\N,Арматурщик,NaN,avito,avito@avito.ru,\N,A-3037298578,\N,1,https://avito.ru/3037298578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9306889,Инженер,Инженер конструктор,Обязанности: Проектирование ультразвуковых пре...,NaN,офисные условия,NaN,35000,35000,NaN,Таганрог,...,\N,Инженер,NaN,avito,avito@avito.ru,\N,A-1580492578,\N,1,https://avito.ru/1580492578
9306890,Монтажник,Монтажник вентиляции/разнорабочий,"Внимание пока обьявление висит, ведется набор!...",Приветствуется опыт по установке систем кондиц...,Оплата производится раз в неделю по субботам!...,NaN,1800,46800,NaN,Краснодар,...,\N,Монтажник,NaN,avito,avito@avito.ru,\N,A-1578246023,\N,1,https://avito.ru/1578246023
9306891,Слесарь-Ремонтник,Слесарь - автомеханик в цех по ремонту карданов,Требуется слесарь в цех по ремонту и балансиро...,Обучаем. Токарные навыки и сварочные навыки пр...,"Зарплата, график работы обсуждаются.",NaN,50000,50000,NaN,Москва,...,\N,Слесарь-Ремонтник,NaN,avito,avito@avito.ru,\N,A-928159469,\N,1,https://avito.ru/928159469


In [11]:
sentences = sentences_df(data)

In [12]:
responsibilities, requirements, terms = sort_respons(sentences)

  0%|          | 0/999 [00:00<?, ?it/s]

In [13]:
#data['responsibilities(Должностные обязанности)'] = responsibilities
data['requirements(Требования к соискателю)'] = requirements
data['terms(Условия)'] = terms

In [14]:
data[['name(название)', 'responsibilities(Должностные обязанности)', 'requirements(Требования к соискателю)', 'terms(Условия)']]

,name(название),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия)
id,,,,
9495846,Арматурщик,Работа на строительных площадках. Очистных соо...,,
9495850,Арматурщик,Гнутье арматурной стали на механических станка...,,
9495851,Арматурщик,Вязка арматуры.,,
2,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,,
9388857,Арматурщик,Вахта в город Москва. Обязанности: - армирова...,Опыт в строительстве приветствуется,Вахта в город москва. Продолжительность вахты ...
...,...,...,...,...
9306889,Инженер,Обязанности: Проектирование ультразвуковых пре...,Знание конструкторских программ приветствуется,
9306890,Монтажник,"Внимание пока обьявление висит, ведется набор!...",Приветствуется опыт по установке систем кондиц...,Оплата производится раз в неделю по субботам. ...
9306891,Слесарь-Ремонтник,Требуется слесарь в цех по ремонту и балансиро...,"Работе на балансировочном, сварочном стенде. Т...","Зарплата, график работы обсуждаются"


In [15]:
data['responsibilities(Должностные обязанности)'] = responsibilities

In [16]:
data

,name(название),specialization(специализация),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия),skills(Ключевые навыки),salary_from,salary_to,object,city,...,updated_by,position,phone,website,email,image,unique_code,city_code,source_id,link_resource
id,,,,,,,,,,,,,,,,,,,,,
9495846,Арматурщик,Арматурщик,,,,NaN,150000,170000,NaN,Димитровград,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-81058502,\N,1,https://hh.ru/vacancy/81058502
9495850,Арматурщик,Арматурщик,Гнутье арматурной стали на механических станка...,,,NaN,130000,150000,NaN,Копейск,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-80163818,\N,1,https://hh.ru/vacancy/80163818
9495851,Арматурщик,Арматурщик,,,,NaN,85000,90000,NaN,Ярославль,...,\N,Арматурщик,NaN,hh,hh@hh.ru,\N,HH-77729491,\N,1,https://hh.ru/vacancy/77729491
2,Арматурщик,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,,,\N,50000,100000,"ООО ""АЛЬМИС_ИНТЕГРАЛ""",\N,...,1,Арматурщик,74959214241,https://almisintegral.ru/,info@almisintegral.ru,\N,\N,\N,1,\N
9388857,Арматурщик,Арматурщик Вахта в мcк (питание+проживание) 60/30,Помощь в прохождение медицинского осмотра,Опыт в строительстве приветствуется,Вахта в город москва. Продолжительность вахты ...,NaN,67000,134000,NaN,Кировск,...,\N,Арматурщик,NaN,avito,avito@avito.ru,\N,A-3037298578,\N,1,https://avito.ru/3037298578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9306889,Инженер,Инженер конструктор,Проектирование ультразвуковых преобразователей...,Знание конструкторских программ приветствуется,,NaN,35000,35000,NaN,Таганрог,...,\N,Инженер,NaN,avito,avito@avito.ru,\N,A-1580492578,\N,1,https://avito.ru/1580492578
9306890,Монтажник,Монтажник вентиляции/разнорабочий,,Приветствуется опыт по установке систем кондиц...,Оплата производится раз в неделю по субботам. ...,NaN,1800,46800,NaN,Краснодар,...,\N,Монтажник,NaN,avito,avito@avito.ru,\N,A-1578246023,\N,1,https://avito.ru/1578246023
9306891,Слесарь-Ремонтник,Слесарь - автомеханик в цех по ремонту карданов,"Снятие установка валов, разборка сборка и бала...","Работе на балансировочном, сварочном стенде. Т...","Зарплата, график работы обсуждаются",NaN,50000,50000,NaN,Москва,...,\N,Слесарь-Ремонтник,NaN,avito,avito@avito.ru,\N,A-928159469,\N,1,https://avito.ru/928159469


Сохранить новый exel файл

In [17]:
with pd.ExcelWriter("Решение.xlsx", engine='xlsxwriter') as writer:
    data[['name(название)', 'responsibilities(Должностные обязанности)', 
          'requirements(Требования к соискателю)', 'terms(Условия)', 'skills(Ключевые навыки)'
         ]].to_excel(writer, sheet_name="Результат")
    writer.sheets['Результат'].set_column(1, 1, 20)
    writer.sheets['Результат'].set_column(2, 4, 60)

### Проверить точность определения Требований + условий (без разделения)

In [18]:
# колонка, полученная с помощью формулы в ячейке в режиме разработчика exel
# только зеленый текст responsibilities(Должностные обязанности)
data_val = pd.read_excel('data/Датасет_разделенный.xlsx', index_col=0)[["name(название)", "no_responsibilities"]]

In [19]:
data_val['no_responsibilities'] = data_val['no_responsibilities'].fillna("")

#### Сравнить с текстом вакансий, выделенным зеленым  
difflib.SequenceMatcher(isjunk=None, a='', b='', autojunk=True)  
isjunk=None - функция, которая фильтрует мусорные элементы,  
a, b - сравниваемые последовательности,  
autojunk=True - отключение автоматической эвристики мусора.  

In [20]:
def get_similarity(data, data_val):
    # определение схожести выделенных требований и условий с размеченными
    idxs = data.index.tolist()
    diffs=[]
    for idx in idxs:
        if (str(data['requirements(Требования к соискателю)'][idx])!="") & (str(data['terms(Условия)'][idx])!=""):
            str_pred = str(data['requirements(Требования к соискателю)'][idx]).lower() +" "+  str(data['terms(Условия)'][idx]).lower()
        elif str(data['requirements(Требования к соискателю)'][idx])!="":
            str_pred = str(data['requirements(Требования к соискателю)'][idx]).lower()
        else:
            str_pred = str(data['terms(Условия)'][idx]).lower()
            
        str_test = str(data_val['no_responsibilities'][idx]).lower()
        #не учитываем знаки припинания и смайлики
        diff = difflib.SequenceMatcher(lambda x: x == " |;|:|.|!|,|\n|[^\x00-\x7Fа-яА-Я]", str_pred, str_test, autojunk=False).ratio() 
        diffs.append(round(diff, 3))
    return np.average(diffs), diffs

In [21]:
mean_diff, list_diff = get_similarity(data, data_val)

In [23]:
mean_diff

0.7182672672672673

In [24]:
temp = data_val[['no_responsibilities']].copy()
temp['requirements(Требования к соискателю)'] = data['requirements(Требования к соискателю)']
temp['terms(Условия)'] = data['terms(Условия)']
temp['similarity']=list_diff

In [70]:
temp[3:12]

,no_responsibilities,requirements(Требования к соискателю),terms(Условия),similarity
id,,,,
2,Выполнение работ по гнутью и резке арматурной ...,,,0.000
9388857,опыт в строительстве приветствуетсяпродолжите...,Опыт в строительстве приветствуется,Вахта в город москва. Продолжительность вахты ...,0.665
9388861,Ставка за смену 3080 руб при графике 6/1 по 11...,Разряда. Опыт работы от 1 года. Наличие удосто...,Предоставляем час обеда + перерывы. Мин. Трудо...,0.683
9388874,опыт в строительстве приветствуется -работа в ...,Опыт в строительстве приветствуется,Продолжительность вахты 60/30 (продление вахты...,0.700
9388888,опыт в строительстве приветствуется; - работа ...,Опыт в строительстве приветствуется,Вахта в город москва. Продолжительность вахты ...,0.689
9496164,,,,1.000
9496155,,,,1.000
9496169,,,,1.000
9336413,Понимание работы в бригаде · Дисциплинированн...,Понимание работы в бригаде. Дисциплинированнос...,"Продолжительность вахты 60/30, 90/30 (можно бо...",0.808


In [26]:
temp[temp['similarity']<0.1]

,no_responsibilities,requirements(Требования к соискателю),terms(Условия),similarity
id,,,,
2,Выполнение работ по гнутью и резке арматурной ...,,,0.000
9336456,,,Вахтовый метод работы 60/30 з/п за месяц. На ...,0.000
9336457,,,Вахтовый метод работы 60/30 з/п за месяц. На ...,0.000
9396310,,Опыт от года. Опыт работы от года,Объект в москве. Премия за выполнение объектов...,0.000
9396406,,"Мы ищем: активных, ответственных людей, с жел...",Мы предлагаем. Стабильную заработную плату (2 ...,0.000
9500588,Отличное техническое оснащение малярного цеха....,,,0.000
14826,,Знание технологических процессов сварки пнд тр...,,0.000
9395175,Вакансия Штукатур в строительстве на чистовую ...,Опыт штукатур / штукатур / шпаклевщик если вы ...,Работа штукатур в москве. Или звоните сами,0.084
9395211,Работа Штукатур в Москве. Вакансия Штукатур в ...,Опыт штукатур / штукатур / шпаклевщик если вы ...,Работа штукатур в москве. Или звоните сами,0.090


In [28]:
def check_str(str1, str2, color):
    # сравнивает одну строку с другой и раскрашивает в случае отличия
    #pattern = r'[^а-яА-ЯёЁa-zA-Z]'
    #str1 = re.sub(pattern, ' ', str1)
    #str1 = re.sub(" +", " ", str1)
    #str2 = re.sub(pattern, '', str2)
    #str2 = re.sub(" +", " ", str2)
    list1 = str1.lower().split()
    list2 = str2.lower().split()
    new_str1 = ""
    for word in list1:
        if word in list2:
            new_str1 += (word + " ")
        else:
            new_str1 += (colored(word, 'black', color) + " ")
    return new_str1

def check_col(df):
    no_resp, req, term = df.columns
    idxs = df.index.tolist()
    col1=[]
    col2=[]
    col3=[]
    for idx in idxs:
        str1 = df[no_resp][idx]
        str2 = df[req][idx]
        str3 = df[term][idx]
        str4 = df[req][idx] + df[term][idx]
        col1.append(check_str(str1, str4, "on_green"))
        col2.append(check_str(str2, str1, "on_yellow"))
        col3.append(check_str(str3, str1, "on_yellow"))
    df = pd.DataFrame({"no_responsibilities": col1, "requirements(Требования к соискателю)": 
                       col2, "terms(Условия)": col3}, index = idxs)
    return df

In [29]:
new_df = check_col(temp[['no_responsibilities', 'requirements(Требования к соискателю)', 'terms(Условия)']])
new_df

,no_responsibilities,requirements(Требования к соискателю),terms(Условия)
9495846,,,
9495850,,,
9495851,,,
2,[42m[30mвыполнение[0m [42m[30mработ[0m ...,,
9388857,опыт в строительстве [42m[30mприветствуетсяп...,опыт в строительстве [43m[30mприветствуется...,[43m[30mвахта[0m в [43m[30mгород[0m [43...
...,...,...,...
9306889,[42m[30mофисные[0m [42m[30mусловия[0m,[43m[30mзнание[0m [43m[30mконструкторских...,
9306890,приветствуется опыт по установке систем [42m...,приветствуется опыт по установке систем кондиц...,оплата производится раз в неделю по [43m[30m...
9306891,[42m[30mобучаем[0m токарные навыки и свароч...,[43m[30mработе[0m [43m[30mна[0m [43m[3...,"[43m[30mзарплата,[0m график работы [43m[3..."
9306892,"опыт работы, знание и понимание принципов рабо...","опыт работы, знание и понимание принципов рабо...",


In [64]:
idxs = new_df.index.tolist()
for idx in idxs[993:]:
    print("\033[1m{}\033[0m".format("Вакансия"), idx)
    print("\033[1m{}\033[0m".format("Текст для распознания: "),  new_df['no_responsibilities'][idx])
    print()
    print("\033[1m{}\033[0m".format("Требования: "),  new_df['requirements(Требования к соискателю)'][idx])
    print("\033[1m{}\033[0m".format("Условия: "),  new_df['terms(Условия)'][idx])
    print()

Вакансия 9306888
Текст для распознания:  отсутствия опыта, обучаеммонтаж воздуховодов, оборудования и систем кондиционирования, 5/2, возможны командировки по башкирии, окладно-премиальная. 

Требования:  стаж работы желательно, в случае отсутствия опыта, обучаем, монтаж воздуховодов, оборудования и систем кондиционирования, 5/2, возможны командировки по башкирии, окладно 
Условия:  

Вакансия 9306889
Текст для распознания:  офисные условия 

Требования:  знание конструкторских программ приветствуется 
Условия:  

Вакансия 9306890
Текст для распознания:  приветствуется опыт по установке систем кондиционирования оплата производится раз в неделю по субботам! только оклад. сделки нет! работа на постоянной основе график работы 6/1, с 8 до 18:00. 

Требования:  приветствуется опыт по установке систем кондиционирования 
Условия:  оплата производится раз в неделю по субботам. только оклад. сделки нет. работа на постоянной основе график работы 6/1, с 8 до 

Вакансия 9306891
Текст для распознани

Поисковый бот

In [253]:
def get_similarity(find, df, n=5):
    # find - запрос соискателя
    # data - таблица с исправленными моделью вакансиями. Бот не будет их считать
    # n - количество выводимых вакансий. закодируй, чтобы красиво было 3-5 штук
    
    #возвращает отфильтвованный датафрейм подходящих вакансий
    data = df.copy()
    idxs = data.index.tolist()
    diffs=[]
    str_find = str(find.lower()) 
    for idx in idxs:
        resp = str(data['responsibilities(Должностные обязанности)'][idx])
        req = str(data['requirements(Требования к соискателю)'][idx])
        term = str(data['terms(Условия)'][idx])
        str_data = resp + " " +  req + " " +  term
        diff = difflib.SequenceMatcher(lambda x: x == " ", str_find, str_data).ratio() 
        diffs.append(round(diff, 3))
        
    data['similarity']=diffs
    #print(data)
    data.sort_values(by='similarity', ascending=False, inplace=True)
   
    # data.head(n)
    return data[0:n]

In [254]:
#загружаем датасет - результат модели (будет файл)
data_itog = pd.read_excel('Решение.xlsx', index_col=0)
# строка, которую вводит соискатель
find = "слесарь сантехник, работа на сварочном стенде"
# отфильстрованный датасет для визуализации
filter_data = get_similarity(find, data_itog)

In [255]:
filter_data

,name(название),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия),skills(Ключевые навыки),similarity
id,,,,,,
9493679,Монтажник,Описание работодателя: монтаж систем скуд скс,Знание систем,NaN,NaN,0.370
9306645,Штукатур,NaN,"Аккуратность, дисциплинированность",Описание работодателя:строительная компания,NaN,0.331
9306839,Монтажник,NaN,NaN,Заработная плата обговаривается непосредственн...,NaN,0.328
9306599,Машинист Крана (Крановщик),Следить за техникой и мелкий ремонт своими силами,NaN,Официальное трудоустройство. Возможны командир...,NaN,0.322
9306804,Монтажник,NaN,NaN,"Оплата труда достойная, своевременная. Спецодежда",NaN,0.314


In [256]:
idxs = filter_data.index.tolist()
idx = idxs[4]
print("Вакансия", filter_data['name(название)'][idx])
print("Должностные обязанности", filter_data['responsibilities(Должностные обязанности)'][idx])
print("Требования к соискателю", filter_data['requirements(Требования к соискателю)'][idx])
print("Условия работы", filter_data['terms(Условия)'][idx])

Вакансия Монтажник
Должностные обязанности nan
Требования к соискателю nan
Условия работы Оплата труда достойная, своевременная. Спецодежда
